In [ ]:
mlist = [
    "distilbert-base-uncased-finetuned-sst-2-english",
    "ProsusAI/finbert", 
    "j-hartmann/emotion-english-distilroberta-base",
    
]

import pandas as pd
from datasets import Dataset, DatasetDict
import mlflow


df_train = pd.read_csv('/home/rechcons/PP3-NLP/age_predict/train.csv',sep="°")
df_test = pd.read_csv('/home/rechcons/PP3-NLP/age_predict/test.csv',sep="°")


train = Dataset.from_pandas(df_train)
test = Dataset.from_pandas(df_test)


dataset = DatasetDict()

dataset['train'] = train
dataset['test'] = test

for x in mlist:
  # %%
  print(x)
  # %%
  from transformers import AutoTokenizer
  model_name = x

  tokenizer = AutoTokenizer.from_pretrained(model_name)
  # %%

  def preprocess_function(examples):
      return tokenizer(examples["text"], truncation=True)

  tokenized_datasets = dataset.map(preprocess_function, batched=True)
  # %%
  from transformers import DataCollatorWithPadding

  data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
  # %%
  from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

  model = AutoModelForSequenceClassification.from_pretrained(model_name, 
                                                            num_labels=6,
                                                            ignore_mismatched_sizes=True)
  # %%
  import numpy as np
  from datasets import load_metric
  metric = load_metric("accuracy")
  #%%
  def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
  # %%
  num_epochs = 2

  args = TrainingArguments(
      output_dir ='./results',          
      num_train_epochs = num_epochs,
      per_device_train_batch_size = 16,   
      per_device_eval_batch_size = 16,   
      weight_decay = 0.01,               
      learning_rate = 2e-5,             
      metric_for_best_model = 'accuracy',    
      evaluation_strategy = "epoch",
      )


  trainer = Trainer(
      model,
      args,
      train_dataset=tokenized_datasets['train'],
      eval_dataset=tokenized_datasets['test'],
      tokenizer=tokenizer,
      data_collator=data_collator,
      compute_metrics=compute_metrics
  )  
  mlflow.autolog()
  trainer.train()
  mlflow.log_param("model",x)

/tmp/ipykernel_65352/3073776217.py:13: ParserWarning: Falling back to the 'python' engine because the separator encoded in utf-8 is > 1 char long, and the 'c' engine does not support such separators; you can avoid this warning by specifying engine='python'.
  df_train = pd.read_csv('/home/rechcons/PP3-NLP/age_predict/train.csv',sep="°")
/tmp/ipykernel_65352/3073776217.py:14: ParserWarning: Falling back to the 'python' engine because the separator encoded in utf-8 is > 1 char long, and the 'c' engine does not support such separators; you can avoid this warning by specifying engine='python'.
  df_test = pd.read_csv('/home/rechcons/PP3-NLP/age_predict/test.csv',sep="°")


distilbert-base-uncased-finetuned-sst-2-english


  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([6, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([6]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023/01/06 16:41:53 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2023/01/06 16:41:53 WARNING mlflow.tracking.fluent: Exception raised while enabling autologging for tensorflow: module 'tensorflow._api.v2.compat.v2.__internal__' has no attribute 'monitoring'
The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.


...

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/j-hartmann/emotion-english-distilroberta-base/resolve/main/config.json from cache at /home/rechcons/.cache/huggingface/transformers/cf0d76ef122b326527bd6a56a127784dd513117bdfde0df5b011c9751bce8fd6.512111953428ded6703782e47e2f67b1c438267f7f4a7c0f19bc1850651a93b4
Model config RobertaConfig {
  "_name_or_path": "j-hartmann/emotion-english-distilroberta-base",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.467531,0.404103
2,No log,1.430400,0.419487


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 3900
  Batch size = 64
/home/rechcons/PP3-NLP/.PPP-NLP-env/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 3900
  Batch size = 64


Training completed. Do not forget to share your model on huggingface.co/models =)


